<a href="https://colab.research.google.com/github/minhnhat232k/Bigdata/blob/main/Assignment2_Bigdata_q5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 11.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612226 sha256=19f8aca8fc180e1405a263c8f6bb883a6524cbdafeaf276c8aaf558d50c4ab43
  Stored in directory: /root/.cache/pip/wheels/19/b0/c8/6cb894117070e130fc44352c2a13f15b6c27e440d04a84fb48
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [18]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType,BooleanType,DateType

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')
spark = SparkSession.builder.getOrCreate()
ratingsDF = spark.read.options(header='True', inferSchema='True', delimiter=',').csv("/content/gdrive/MyDrive/Bigdata-Pyspark-Dataset/Assignment2/ratings_small.csv")
moviesDF = spark.read.options(header='True', inferSchema='True', delimiter=',').csv("/content/gdrive/MyDrive/Bigdata-Pyspark-Dataset/Assignment2/movies_small.csv")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
moviesDF = moviesDF.withColumn("genres_array", f.split("genres", "\|")) \
.withColumn('last_genres', f.element_at("genres_array", -1))


In [21]:
user_genre_ratings = ratingsDF.join(moviesDF, on='movieId', how='left') \
.groupBy('userId', 'last_genres') \
.agg(f.avg('rating').alias('avg_rating'))

In [22]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col,to_date,date_format

window_spec = Window.partitionBy('userId').orderBy(col('avg_rating').desc())

user_top_genre = user_genre_ratings.withColumn('rank', f.row_number().over(window_spec)) \
.where(col('rank') == 1) \
.select('userId', 'last_genres')

window_spec_movies = Window.partitionBy('last_genres').orderBy(col('avg_rating').desc())

result_q5 = user_top_genre.join(moviesDF, on='last_genres', how='left') \
  .join(user_genre_ratings, on=['userId', 'last_genres'], how='left') \
  .withColumn('rank',f.row_number().over(window_spec_movies)) \
  .where(col('rank') <= 5) \
  .groupBy('userId') \
  .agg(f.max('last_genres').alias('Highest_rated_genre_name'), \
  f.collect_list('title').alias('Top_5_unrated_movies_with_highest_rating'))


In [23]:
result_q5 = result_q5.orderBy('userId')
result_q5.show(100, truncate=False)


+------+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|Highest_rated_genre_name|Top_5_unrated_movies_with_highest_rating                                                                                                                                                                      |
+------+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|9     |Adventure               |[The Legend of Tarzan (2016), Neon Bull (2015), Return of the One-Armed Swordsman (1969), God Loves Caviar (2012), Last Knights (2015)]                                                                       |
|30    |War                     |[Da